In [78]:
# import libraries
import numpy as np
from sklearn.neighbors import KNeighborsClassifier

##Ejercicio 1

In [79]:
### a

# obtain the headers
with open('wine.csv', 'r') as file:
    headers = file.readline().split(',')

# read the data from the csv file
data = np.genfromtxt('wine.csv', delimiter=',')

# drop the text headers and keep only the values
data = data[1:]

In [80]:
print(data.shape)
print(data[5])

(178, 14)
[1.42e+01 1.76e+00 2.45e+00 1.52e+01 1.12e+02 3.27e+00 3.39e+00 3.40e-01
 1.97e+00 6.75e+00 1.05e+00 2.85e+00 1.45e+03 1.00e+00]


In [81]:
### b

# get the independant variables
x = data[:, :-1]

# get the single dependant variable
y = data[:, -1]

In [82]:
### c

# obtain the mean vector of data
def get_mean_vector(x):
    return np.mean(x, axis=0)

# obtain the standard deviation vector of data
def get_std_vector(x):
    rows_qty = x.shape[0]
    col_qty = x.shape[1]
    ret = np.zeros(col_qty)

    mean = get_mean_vector(x)

    for i in range(rows_qty):
        ret = ret + (x[i] - mean) ** 2

    return ret / (rows_qty - 1)

# obtain the normalized vector of data
def get_normailized(x):
    return (x - get_mean_vector(x)) / get_std_vector(x)

x = get_normailized(x)

In [83]:
### d

cov = np.cov(x, rowvar=False)

In [84]:
### e

def get_max_eigenvector(matrix, iter = 20):
    # start with a random vector
    ret = np.random.rand(matrix.shape[1])

    # apply the power method, simply multiplying the matrix with the vector and dividing by the norm
    for _ in range(iter):
        next = np.dot(matrix, ret)
        ret = next / np.linalg.norm(next)
    return ret

# obtain the eigenvalue simply by calculating using the formula see in class
def get_related_eigenvalue(matrix, max_eigenvector):
    t_max_eigenvector = np.transpose(max_eigenvector)
    return np.dot(t_max_eigenvector, np.dot(matrix, max_eigenvector)) / (np.dot(t_max_eigenvector, max_eigenvector))

max_cov_eigenvector = get_max_eigenvector(cov)
max_cov_eigenvalue = get_related_eigenvalue(cov, max_cov_eigenvector)


In [85]:
### f

def get_max_eigenvectors(matrix, qty = 1, iter = 200):
    
    eigenvectors = np.zeros(shape=(qty, matrix.shape[1]))

    for i in range(qty):
        aux = np.random.rand(matrix.shape[1])

        # for the next iteration start with a random vector and remove the components of already calculed eigenvectors
        for _ in range(iter):
            for eigenvector in eigenvectors:
                aux = aux - np.dot(eigenvector, aux) * eigenvector
            next_aux = np.dot(matrix, aux)
            aux = next_aux / np.linalg.norm(next_aux)

        eigenvectors[i] = aux

    return eigenvectors

def get_max_eigenvalues(matrix, eigenvectors):
    eigenvalues = np.zeros(eigenvectors.shape[0])

    i = 0
    for eigenvector in eigenvectors:
        eigenvalues[i] = get_related_eigenvalue(matrix, eigenvector)
        i = i + 1
        
    return eigenvalues

eigenvectors = get_max_eigenvectors(cov, cov.shape[0])
eigenvalues = np.array(get_max_eigenvalues(cov, eigenvectors))

# print(np.allclose(np.linalg.eig(cov).eigenvalues, eigenvalues, rtol=1e-03))
# print(np.allclose(np.transpose(np.linalg.eig(cov).eigenvectors  ** 2), eigenvectors  ** 2, rtol=1e-03))

In [86]:
### g

# predict the result label of the input array containing the samples atributes and needs the eigenvectors of the covariance matrix
def predict_data(input, eigenvectors):

    # transform the data into the new basis using the eigenvectors
    transformed_x = np.dot(x, np.transpose(eigenvectors))
    
    knn = KNeighborsClassifier(n_neighbors=5)

    # fit the transformed data in to the KNN model
    knn.fit(transformed_x, y)

    # predit the result transforming the input data into the new basis and then using the knn model
    return knn.predict(np.dot(input, np.transpose(eigenvectors)))

# take all the samples of the data which their index is divisible by 10 for testing the model
test = x[::10]
correct_labels = y[::10]
predicted_data = predict_data(test, eigenvectors)

for i in range(len(predicted_data)):
    print (", actual is: " + str(correct_labels[i]) + "prediction is: " + str(predicted_data[i]))


, actual is: 1.0prediction is: 1.0
, actual is: 1.0prediction is: 1.0
, actual is: 1.0prediction is: 1.0
, actual is: 1.0prediction is: 1.0
, actual is: 1.0prediction is: 1.0
, actual is: 1.0prediction is: 1.0
, actual is: 2.0prediction is: 2.0
, actual is: 2.0prediction is: 2.0
, actual is: 2.0prediction is: 2.0
, actual is: 2.0prediction is: 2.0
, actual is: 2.0prediction is: 2.0
, actual is: 2.0prediction is: 2.0
, actual is: 2.0prediction is: 1.0
, actual is: 3.0prediction is: 3.0
, actual is: 3.0prediction is: 3.0
, actual is: 3.0prediction is: 3.0
, actual is: 3.0prediction is: 3.0
, actual is: 3.0prediction is: 3.0


##Ejercicio 2

In [90]:
### a

from sklearn.model_selection import train_test_split


# predict the result label of the input array containing the samples atributes and needs the eigenvectors of the covariance matrix
def benchmark(x, y, eigenvectors, eigenvalues):

    # make a shuffled training and test set
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=11)
    
    #transform the training data into the new basis using the eigenvectors
    transformed_x_train = np.dot(x_train, np.transpose(eigenvectors))

    # for the case when only use the first principal component 
    # if the training data is a vector, transform it into a matrix with one column
    if (transformed_x_train.shape == (transformed_x_train.shape[0],)):
        transformed_x_train = transformed_x_train.reshape(-1, 1)

    knn = KNeighborsClassifier(n_neighbors=5)

    # fit the transformed data in to the KNN model
    knn.fit(transformed_x_train, y_train)

    #transform the training data into the new basis using the eigenvectors
    transformed_x_test = np.dot(x_test, np.transpose(eigenvectors))

    # for the case when only use the first principal component 
    # if the training data is a vector, transform it into a matrix with one column
    if (transformed_x_test.shape == (transformed_x_test.shape[0],)):
        transformed_x_test = transformed_x_test.reshape(-1, 1)

    # predit the result transforming the test data into the new basis and then using the knn model
    predicted_y = knn.predict(transformed_x_test)

    # calculate the accuracy
    accuracy = np.sum(predicted_y == y_test) / len(y_test)

    print(transformed_x_train.shape[1])

    explain_varianze = eigenvalues[transformed_x_train.shape[1] - 1] / np.sum(eigenvalues)
    percen_explain_varianze = 100 * explain_varianze
    accumulated_explain_varianze = np.sum(eigenvalues[0:transformed_x_train.shape[1]]) / np.sum(eigenvalues)

    return accuracy, explain_varianze, percen_explain_varianze, accumulated_explain_varianze

print(benchmark(x, y, eigenvectors[0, :], eigenvalues))
print(benchmark(x, y, eigenvectors[[0,1], :], eigenvalues))
print(benchmark(x, y, eigenvectors[[0,1,2], :], eigenvalues))
print(benchmark(x, y, eigenvectors[[0,1,2,3], :], eigenvalues))

1
(0.6111111111111112, 0.6391973009097871, 63.91973009097871, 0.6391973009097871)
2
(0.75, 0.17048331542044542, 17.048331542044544, 0.8096806163302325)
3
(0.8333333333333334, 0.12202397400868929, 12.202397400868929, 0.9317045903389217)
4
(0.8888888888888888, 0.03255070436982658, 3.255070436982658, 0.9642552947087484)
